In [1]:
import os
import re
from tqdm.auto import trange
from urllib.request import urlopen
from urllib.error import HTTPError
from urllib.parse import urljoin
from bs4 import BeautifulSoup

In [2]:
# Add the desired Categories in list categories_to_choose

#    'Acne-and-Rosacea-Photos'
#    'Actinic-Keratosis-Basal-Cell-Carcinoma-and-other-Malignant-Lesions'
#    'Atopic-Dermatitis-Photos'
#    'Bullous-Disease-Photos'
#    'Cellulitis-Impetigo-and-other-Bacterial-Infections'
#    'Eczema-Photos'
#    'Exanthems-and-Drug-Eruptions'
#    'Hair-Loss-Photos-Alopecia-and-other-Hair-Diseases'
#    'Herpes-HPV-and-other-STDs-Photos'
#    'Light-Diseases-and-Disorders-of-Pigmentation'
#    'Lupus-and-other-Connective-Tissue-diseases'
#    'Melanoma-Skin-Cancer-Nevi-and-Moles'
#    'Nail-Fungus-and-other-Nail-Disease'
#    'Poison-Ivy-Photos-and-other-Contact-Dermatitis'
#    'Psoriasis-pictures-Lichen-Planus-and-related-diseases'
#    'Scabies-Lyme-Disease-and-other-Infestations-and-Bites'
#    'Seborrheic-Keratoses-and-other-Benign-Tumors'
#    'Systemic-Disease'
#    'Tinea-Ringworm-Candidiasis-and-other-Fungal-Infections'
#    'Urticaria-Hives'
#    'Vascular-Tumors'
#    'Vasculitis-Photos'
#    'Warts-Molluscum-and-other-Viral-Infections'


categories_to_choose = [
    'Acne-and-Rosacea-Photos',
    'Atopic-Dermatitis-Photos',
    'Bullous-Disease-Photos',
    'Cellulitis-Impetigo-and-other-Bacterial-Infections',
    'Eczema-Photos',
    'Herpes-HPV-and-other-STDs-Photos',
    'Light-Diseases-and-Disorders-of-Pigmentation',
    'Lupus-and-other-Connective-Tissue-diseases',
    'Nail-Fungus-and-other-Nail-Disease',
    'Psoriasis-pictures-Lichen-Planus-and-related-diseases',
    'Scabies-Lyme-Disease-and-other-Infestations-and-Bites',
    'Seborrheic-Keratoses-and-other-Benign-Tumors',
    'Tinea-Ringworm-Candidiasis-and-other-Fungal-Infections',
    'Warts-Molluscum-and-other-Viral-Infections'
]
len_categories = len(categories_to_choose)

In [3]:
DERMNET_CATEGORIES = "http://www.dermnet.com/dermatology-pictures-skin-disease-pictures/"
DERMNET = "http://www.dermnet.com/"
DERMNET_IMAGES = "http://www.dermnet.com/images/"

In [4]:
def getURL():
# Returns a dictionary of URLs of images to download.
    
    def CategoryURLs(url):
    # Returns list of all Image URLs of a Category
        images = []
        soup = soupify(url)
        links = soup.find("table").find_all("a")
    
        cat_urls = []
        for l in links:
            link = urljoin(DERMNET, l.get('href'))
            cat_urls.append(link)

        print('> Number of Sub-Categories : {}.'.format(len(cat_urls)))

        for i, u in enumerate(cat_urls):
            print('-> Fetching URLs from Sub-Categories {}/{}'.format(i + 1, len(cat_urls)))
            images.extend(SubCategoryURLs(u))
    
        return images
    
    def SubCategoryURLs(category_url):
    # Returns list of Image URLs of a Sub-Category
        images = []
        soup = soupify(category_url)
        thumbnails = soup.find_all("div","thumbnails")
        if thumbnails:
            for thumb in thumbnails:
                thumb_link = thumb.img['src']
                image_link = re.sub(r'Thumb',"",thumb_link)
                images.append(image_link)
    
        soup = soupify(category_url)
        pages = soup.find("div", "pagination")
        thumb_urls = []
    
        if pages:
            for page in pages:
                if page.name == 'a' and page.string != 'Next':
                    page_url = urljoin(DERMNET, page['href'])
                    thumb_urls.append(page_url)

        for page in thumb_urls:
            soup = soupify(page)
            thumbnails = soup.find_all("div","thumbnails")
            if thumbnails:
                for thumb in thumbnails:
                    thumb_link = thumb.img['src']
                    image_link = re.sub(r'Thumb',"",thumb_link)
                    images.append(image_link)
    
        return images

    def soupify(url):
    # BeautifulSoup's lxml HTML parser
        html = urlopen(url)
        soup = BeautifulSoup(html, "lxml")
        return soup
    
    soup = soupify(DERMNET_CATEGORIES)
    cat_links = soup.find("table").find_all("a")
    n_links = len(categories_to_choose)
    print("Number of Categories : {}.".format(n_links))
    n_total = 0
    all_categories = []
    url_dict = {}
    links = []
    final_links = []
    for link in cat_links:
        links.append(urljoin(DERMNET, link.get('href')))
    len_all = len(links)
    for i in range(len_all):
        for j in range(len_categories):
            if(categories_to_choose[j] == links[i].split(DERMNET_IMAGES)[1].split('/')[0]):
                final_links.append(links[i])
 
    for i in range(len_categories):
        cat_name = re.sub(r'[^a-z0-9A-Z\s]+', ' ', final_links[i].split(DERMNET_IMAGES)[1].split('/')[0])
        print('\nFetching URLs for Category {}/{} : {}\n'.format(i + 1, len_categories, cat_name))
        cat_images = CategoryURLs(final_links[i])
        n_total += len(cat_images)
        print('\nNumber of fetched images : {}. Total images : {}.\n'.format(len(cat_images), n_total))
        url_dict[cat_name] = cat_images

    return url_dict


In [5]:
urls = getURL()

Number of Categories : 14.

Fetching URLs for Category 1/14 : Acne and Rosacea Photos

> Number of Sub-Categories : 25.
-> Fetching URLs from Sub-Categories 1/25
-> Fetching URLs from Sub-Categories 2/25
-> Fetching URLs from Sub-Categories 3/25
-> Fetching URLs from Sub-Categories 4/25
-> Fetching URLs from Sub-Categories 5/25
-> Fetching URLs from Sub-Categories 6/25
-> Fetching URLs from Sub-Categories 7/25
-> Fetching URLs from Sub-Categories 8/25
-> Fetching URLs from Sub-Categories 9/25
-> Fetching URLs from Sub-Categories 10/25
-> Fetching URLs from Sub-Categories 11/25
-> Fetching URLs from Sub-Categories 12/25
-> Fetching URLs from Sub-Categories 13/25
-> Fetching URLs from Sub-Categories 14/25
-> Fetching URLs from Sub-Categories 15/25
-> Fetching URLs from Sub-Categories 16/25
-> Fetching URLs from Sub-Categories 17/25
-> Fetching URLs from Sub-Categories 18/25
-> Fetching URLs from Sub-Categories 19/25
-> Fetching URLs from Sub-Categories 20/25
-> Fetching URLs from Sub-Cat

In [6]:
# Now we Download the images

OUT_DIRECTORY = '/kaggle/working/'
n_images = 0
for cat_name, images in urls.items():
    n_images += len(images)

n_downloaded = 0
for cat_name, images in urls.items():
    cat_path = os.path.join((OUT_DIRECTORY), '-'.join(cat_name.split()))
    if not os.path.exists(cat_path):
        os.mkdir(cat_path)
    print(cat_name)
    for i in trange(len(images)):
        if(' ' not in images[i]):
            image_name = os.path.basename(images[i])
            file_name = os.path.join(cat_path, image_name)
            try:
                f = urlopen(images[i]).read()
                open(file_name, 'wb').write(f)
                n_downloaded += 1
            except HTTPError:
                continue
print('Downloaded {} images.'.format(n_downloaded))

Acne and Rosacea Photos



Atopic Dermatitis Photos



Bullous Disease Photos



Cellulitis Impetigo and other Bacterial Infections



Eczema Photos


...


Scabies Lyme Disease and other Infestations and Bites



Seborrheic Keratoses and other Benign Tumors



Tinea Ringworm Candidiasis and other Fungal Infections



Warts Molluscum and other Viral Infections



Downloaded 14017 images.
